### 딕셔너리 삽입순서에 의존할 때 조심하라

* 파이썬 3.7부터는 dict  인스턴스 내용이 이터레이션 될 때 키를 삽입한 순서대로 된다.
* 파이썬 딕셔너리와 유사한 객체가 있는데 이런 타입의 경우 키 삽입 순서가 보존되는 것은 아니다.
* 딕셔너리와 비슷한 클래스를 조심스럽게 다루기 위해서 dict 인스턴스의 삽입순서 보존에 의존하지 않는 코드를 작성한다.

### 선행 지식
* dictionary 구조
* ```__dict__``` 사용하는 때와 사용법
* list sort : names.sort(key=votes.get, reverse=True)
* enumerate
* 딕셔너리 프코토콜을 따르는 사용자 정의 딕셔너리 만드는 법
* instance 확인하는 법: isinstance
* raise typeerror 만드는 법
* MutableMapping 상속받아서 사용하는 법

파이썬 3.5 이전에서는 딕셔너리 키 삽입 순서에 의존하지 않았다.
파이썬 3.6 부터는 삽입 순서를 보존하도록 동작이 개선되었다.

In [1]:
# python 3.5에서는 다른 순서로 나온다.
# 3.5 이후 버전에서만 아래와 같이 입력한 순서로 나온다.
baby_names={
    'cat': 'kitten',
    'dog': 'puppy',
}

print(baby_names)

{'cat': 'kitten', 'dog': 'puppy'}


key, value, items, popitem 등 딕셔너리가 제공하는 메소드도 파이썬 3.7부터는 순서에 의존할수 있게 되었다.

In [6]:
# 3.5 이하에서는 아래와 같은 순서가 유지되지 않았다.
# 이후 버전에서는 입력한 순서를 유지한다.
print(list(baby_names.keys()))
print(list(baby_names.values()))
print(list(baby_names.items()))
print(baby_names.popitem())

['cat', 'dog']
['kitten', 'puppy']
[('cat', 'kitten'), ('dog', 'puppy')]
('dog', 'puppy')


함수에 대한 키워드 인자를 다루는 경우도 순서가 보존된다
3.5 이전에는 키워드 인자의 순서가 뒤죽박죽 이었다.
함수 키워드 인자를 출력하는 예제

In [19]:
def my_func(**kwargs):
    for key, value in kwargs.items():
        print(f'{key} = {value}')

my_func(goose = 'gosling', kangaroo='joey')

goose = gosling
kangaroo = joey


함수 뿐만 아니라 인스턴스 필드를 나타내는 __dict__  변수도 삽입한 순서를 유지하는 것을 확인할 수 있다.
여기서 __dict__는 클래스 인스턴스 변수를 담고 있는 딕셔너리이다.

In [5]:
class MyClass:
    def __init__(self):
        self.alligator = 'hatchiling'
        self.elephant = 'calf'

a = MyClass()
for key, value in a.__dict__.items():
    print(f'{key} = {value}')

alligator = hatchiling
elephant = calf


딕셔너리를 처리할 때 삽입순서 동작이 항상 성립한다고 가정해서는 안된다. 파이썬은 list, dict등의 표준 프로토콜을 흉내내는 커스텀 컨테이너 타입을 쉽게 정의할 수 있다. 파이썬은 동적으로 타입이 결정되는 덕타이핑에 의존하며 이로인해 가끔 어려운 함정에 빠질 수 있다.

가장 귀여운 아기 동물을 뽑는 콘테스트의 결과를 보여주는 프로그램을 작성해 보자.
득표수를 저장한 딕셔너리를 만들고 아기동물의 이름을 따로 뽑아서 리스트로 저장한 다음에 아기동물의 이름을 득표수로 sorting 한다.


In [16]:
votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox':863,
}

In [17]:
names = list(votes.keys())
print(names)

names.sort(key=votes.get, reverse=True)
print(names)

['otter', 'polar bear', 'fox']
['otter', 'fox', 'polar bear']


최종 프로그램은 아래와 같다.

In [18]:
# votes에 따라 정렬하고 winner를 출력하는 프로그램을 만들라
votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox':863,
}

ranks={}

def populate_ranks(votes, ranks):
    # votes를 value 값으로 sort
    # sort결과를 ranks에 넣어준다.
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True) #get은 key에 해당하는 value
    for i, name in enumerate(names, 1):
        ranks[name]=i

def get_winner(ranks):
    return next(iter(ranks))


populate_ranks(votes, ranks)
print(ranks)
winner = get_winner(ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
otter


프로그램의 요구사항이 변경되어서 등수가 아니라 알파벳 순으로 보여준다고 가정해보자
이때, collections.abc를 이용해 딕셔너리와 비슷하지만 내용을 알파벳 내용을 알파벳 순서대로 이터레이션하는 클래스를 만든다.
딕셔너리와 비슷한 클래스를 만든다는 것은 딕셔너리의 표준 프로토콜을 지킨다는 의미이다.
즉, ```__getitem__, __setitem__, __delitem__, __iter__, __len__```을 정의하는 클래스를 만드는 것이다.
이렇게 만든 사용자 정의 딕셔너리는 앞에서 만든 populate_ranks를 호출해도 아무런 문제가 생기지 않는다.
이렇게 만든 클래스를 이용해 winner를 출력했는데 1등이 출력되지 않고 알파벳 순서로 맨 앞에 있는 fox가 출력되었다. 왜 이런 문제가 발생하는가?

In [21]:
# 요구사항 변경: 등수가 아니라 알파벳 순으로 보여줄 것
# 알파벳 순서대로 이터레이션하는 클래스 작성
from collections.abc import MutableMapping

class SortedDict(MutableMapping):
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        self.data[key] = value

    def __delitem__(self,key):
        del self.data[key]

    def __iter__(self):
        keys = list(self.data.keys())
        keys.sort()
        for key in keys:
            yield key

    def __len__(self):
        return len(self.data)

# 문제점: 1등을 뽑아주지 못함
# 알파벳 순서대로 iteration 하였기 때문이다.
def get_winner(ranks):
    return next(iter(ranks))

sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
fox


위와 같은 문제가 발생한 이유는 get_winner를 구현할때 암묵적으로 사용자 딕셔너리 (SortedDict)가 사용자가 삽입한 순서로 이터레이션 하고 있다고 가정하기 때문이다. 그러나 `__iter__`  함수에서 sorting을 key값 기준으로 정렬된 순서로 이터레이션 하도록 구현되어 있어서 문제가 발생한 것이다. 이러한 문제를 해결하는 방법은?

첫번째 방법은 ranks (여기서는 sorteddict) 딕셔너리가 어떤 순서로 이터레이션 된다고 가정하지 않고 get_winner 함수를 구현하는 것이다. 가장 보수적인 방법이다.

In [25]:
# 다음과 같이 데이터가 저장되어 있음을 확인해보자!!
list(sorted_ranks.items())

[('fox', 2), ('otter', 1), ('polar bear', 3)]

In [24]:
# 방법1: 1등인 rank를 뽑는다.
# 문제점: 전체를 다 돌아야 하는 단점이 있다.
def get_winner(ranks):
    for name, rank in ranks.items():
        if rank ==1:
            return name

winner = get_winner(sorted_ranks)
print(winner)

otter


방법 2는 ranks가 우리가 원하는 타입일때만 실행시킨다. 이 방법은 보수적인 접근보다는 실행 성능이 우수하다.
우리가 원하는 타입 검사는 isinstance 이다.

In [26]:
# 방법2: dict일때만 실행하게 한다.
def get_winner(ranks):
    if not isinstance(ranks, dict):
        raise TypeError('dict 인스턴스가 필요합니다.')
    return next(iter(ranks))

get_winner(sorted_ranks)

TypeError: dict 인스턴스가 필요합니다.

In [29]:
# ranks가 dict 이기 때문에 우리가 원하는 otter가 출력이 된다.
get_winner(ranks)

'otter'

세번째 방법은 Type annotation을 사용하는 해서 get_winner에 전달되는 값을 dict 인스턴스가 되도록 강제하는 것이다.
세번째 방법은 mypy를 사용해서 타입 체크를 하는 것인데 생각보다 사용하는 방법이 쉽지 않아서 아래 내용은 스킵한다.

In [37]:
from typing import Dict, MutableMapping
def populate_ranks(votes: Dict[str, int],
                   ranks: Dict[str, int]) -> None:
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

def get_winner(ranks: Dict[str, int]) -> str:
    return next(iter(ranks))

sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)

winner = get_winner(sorted_ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
fox
